Target: 

```bash
echo "{\"brand\": \"ol'name\"}"
```

Si single quoted: impossible
Si double quoted: all double quotes must be escaped

```bash
echo "\"hello\"" # -> "hello"
```

```bash
echo '\"hello\"' # -> \"hello\"
```

quoting = remove meaning of special chars/words = prevent special treatments/reserved words reco/expansion. Ex: double quotes can be quoted inside double quotes if escaped with `\`. Single quotes however, cannot be quoted (i.e. escaped) within single quotes.
* backslash (= remove special meaning of a single char): bash escape (=preserve literal value of next char) char
* single quote (=remove all special meaning/interpretation of a sequence of char (single quote excluded)): preserve literal value of all enclosed chars, single quotes not being allowed
* double quote (=remove most special meaning/interpretation of a sequence of char): Preserve literal value of all chars, backtick, `$` `\` (backslash: keep special meaning - i.e. are removed - if followed by backtick, `$`, `"`, `\` or `newline`) meaning that expansion mechanisms remain, and depending of the config, `!` 

Exception object
`__context__`
`__suppress_context__` => `bool`
`__cause__`
`__traceback__` : `traceback` object which encapsulates the call stack at the point where the exception originally occurred.
`with_traceback`
`args` => correspond aux arguments (forcément sans keyword) passés au constructeur de l'exception.

https://julien.danjou.info/python-exceptions-guide/
https://luminousmen.com/post/context-vs-cause-attributes-in-exception-handling
https://www.python.org/dev/peps/pep-3134/
https://www.python.org/dev/peps/pep-0409/
https://www.teclado.com/30-days-of-python/python-30-day-24-exceptions-advanced
https://blog.ram.rachum.com/post/621791438475296768/improving-python-exception-chaining-with
https://stefan.sofa-rockers.org/2020/10/28/raise-from/
https://stackoverflow.com/questions/17015230/are-nested-try-except-blocks-in-python-a-good-programming-practice

`sys.exc_info()` => Returns a `(type, value, traceback)` tuple that gives informations about the exception that is currently being handled. Here, “handling an exception” is defined as “executing an except clause.” The information returned is specific both to the current thread and to the current stack frame. If the current stack frame is not handling an exception, the information is taken from the calling stack frame, or its caller, and so on until a stack frame is found that is handling an exception. If no exception is being handled anywhere on the stack, a tuple containing three None values is returned.
* `type` gets the type ("class object") of the exception being handled (a subclass of `BaseException`)
* `value` gets the exception instance (an instance of the `type`)
* `traceback` gets a `traceback` object which encapsulates the call stack at the point where the exception originally occurred.

In [ ]:
class MyContextManager:
    
    def __init__(self, handle_error):
        self._handle_error = handle_error
    
    def __enter__(self):
        # Can return an object built in this method (connection, etc.)
        pass
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f'__exit__({exc_type}, {exc_val}, {exc_tb})')
        if exc_type:
            print(f"Raised exception: {exc_type.__name__}")
        return self._handle_error

### `__exit__` method
If no exception occured in the block nested in the `with` statement is executed, all three arguments will be `None`. Otherwise, an exception is supplied. What happens next only depends on the value returned by the `__exit__` method. If the method returns `True` then the execption will not be reraised after exiting the `__exit__` method. We say that the exception has been suppressed and that we prevented its propagation. Otherwise, the exception is automatically reraised after exiting the method, the exception is being propagated. The `__exit__` method must therefore **not reraise** the passed-in exception.

In [ ]:
with MyContextManager(True):
    raise Exception('Unhandled exception')

In [ ]:
with MyContextManager(False):
    raise Exception('Unhandled exception')

Creating context managers the traditional way, by writing a class with `__enter__` and `__exit__` methods, is not difficult. But sometimes, some people consider it is more overhead than you need just to manage a trivial bit of context. In those sorts of situations, you can convert a generator function into a context manager using a dedicated decorator: `contextlib.contextmanager`.

The `yield` statement of the generator function (which should yield **exactly once**) splits the function body into two parts. The section that precedes the `yield` statement is dedicated to initializing the context and plays a similar role as the `__enter__` method. The `yield` statement yields the control to the block within the `with` statement. The value yielded, if any, is bound to the variable in the `as` clause of the with statement. At the point where the generator yields, the block nested in the `with` statement is executed. The generator function is then reentered, the section following the `yield` statement being dedicated to closing the context. This section plays a similar role as the `__exit__` method.

Uncaught exceptions from the `with` block are reraised right after reentry in the generator function where they can be handled (which requires the `yield` statement to be part of a `try...except...finally` block. The generator function may not be the appropriate location to handle the error, but catching it using a `try...except` construct allows at least to make sure that the context is closed properly.     

If the exception is handled by the generator function, it indicates to the `with` statement that the handling took place and execution resumes immediately following the `with` statement. If for any reason (log, perform complex logic, etc.) we want the exception to be handled outside from the generator function, the function must reraise the exception (after performing the context closing logic).

In [ ]:
import contextlib

@contextlib.contextmanager
def make_context():
    print('Perform context initialization logic')
    context_obj = dict()
    try:
        yield context_obj
    except RuntimeError:
        print('Exception handling logic')
    finally:
        print('Perform context closing logic')

In [ ]:
with make_context() as context:
    raise RuntimeError('Handled exception')

In [ ]:
with make_context() as context:
    raise AttributeError('Unhandled exception')

Reminder: If a `finally` clause is present, the `finally` clause will execute as the last task before the `try` statement completes. The `finally` clause runs whether or not the `try` statement produces an exception. If an exception occurs during execution of the `try` clause, the exception may be handled by an `except` clause. We are garanteed that the `finally` clause always executes. In particular, if an exception is not handled by an `except` clause, the exception is automatically re-raised after the `finally` clause has been executed. An exception could occur during execution of an `except` or `else` clause. Again, the exception is automatically re-raised after the `finally` clause has been executed.

In [ ]:
@contextlib.contextmanager
def make_context():
    print('Perform context initialization logic')
    context_obj = dict()
    try:
        yield context_obj
    except Exception as e:
        print('Exception is just caught, it is re-raised for further handling')
        raise e
    finally:
        print('Perform context closing logic')

In [ ]:
with make_context() as context:
    raise Exception('Unhandled exception')